# Facial_key_points

### IMPORT PACKAGES

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import os


# os.chdir('/Users/romankasichhwa/Desktop/Facial_key_points') #manage path accordingly!
# print(os.getcwd())

In [45]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

### HYPERPARAMETERS

In [34]:
batch_size = None
model_input_size = 224


### DATALOADER

In [55]:
class FacialKeyPointsDataset(Dataset):
    def __init__(self, csv_file_path = r'data/training_frames_keypoints.csv', split = 'training',device=torch.device('cpu')):  
        super().__init__()
        self.csv_file_path = csv_file_path
        self.split = split
        self.df = pd.read_csv(self.csv_file_path)
        # print(self.df)    -->1
        self.normalize = transforms.Normalize(
            mean=[0.485, 0.456, 0.406],   #[r,g,b]
            std=[0.229, 0.224, 0.225]
        )
        self.device = device
        
        
    def __len__(self):
        return len(self.df)
    
    
    def __getitem__(self, idx):
        img, original_size = self.get_img(idx)
        facial_keypoints = self.get_keypoints(index = idx, original_size = original_size)
        return img, facial_keypoints
    
    
    def get_img(self, index):    # to get img from  training folder using the keypoints df
        # print(self.df.iloc[index,0])   # --> 3
        # print(os.path.join(os.getcwd(), 'data', self.split, self.df.iloc[index,0]))   # ---> 4
        img_path = os.path.join(os.path.join(os.getcwd(), 'data', self.split, self.df.iloc[index,0]))
        img = Image.open(img_path).convert('RGB')
        original_size = img.size
        # print(original_size)    # ---> 5
        
        #pre-process image  ( normalizing and then converting to tensors)
        img = img.resize((model_input_size, model_input_size))
        # print(img.size)
        # print(np.asarray(img))
        img = np.asarray(img)/255.0  #range of pixel value is between 0(black) and 255(white), we normalize it to [0,1]
        # print(img.shape)  ---> (224,224,3)  i.e ( height , width, channels(RGB))
        
        
        #but in pytorch image_tensor should nbe represented in standard form ( batch,channel, width,height), to solve it we use permute 
        img = torch.tensor(img).permute(2,0,1).unsqueeze(0)
        # print(img.shape)    --> torch.Size([1, 3, 224, 224])
        img = self.normalize(img)
        # print(img)
        return img.to(self.device), original_size
    
    
    
    def get_keypoints(self, index,original_size):
        kp = self.df.iloc[index, 1:].to_numpy().astype(np.float32)  # since kp_x,y is list ,we cant divide it by int . So, we use broadcasting concept in numpy
        kp_x = kp[0::2] / original_size[0]
        kp_y = kp[1::2] / original_size[1]
        kp = np.concatenate([kp_x, kp_y])  # required ip to the model 
        print(type(kp))
        # print(kp)
        return torch.tensor(kp).to(self.device)
    
    
    def load_img(self, index):
        img_path = os.path.join(os.getcwd(), 'data', self.split, self.df.iloc[index,0])
        img = Image.open(img_path).convert('RGB')
        img = img.resize((model_input_size, model_input_size))
        return np.asarray(img) / 255.0
        
        
        
        
        
    
    
training_data = FacialKeyPointsDataset(device=device)
# training_data[0]   #-->2
test_data = FacialKeyPointsDataset(csv_file_path=r'data/test_frames_keypoints.csv', split='test', device=device)

/Users/romankasichhwa/Desktop/Facial_key_points/notebook
